### Building the dataset

When creating a neural network to play chess, one of the fundamental steps is to build a comprehensive dataset that comprises chess board positions and corresponding moves. This dataset can be utilized to train the neural network to predict the optimal move for a given position.

To build the dataset, one can use a chess engine to produce numerous chess games and document the board positions and moves executed in each game. A Python chess library such as python-chess can help in achieving this objective.

Upon acquiring the dataset, the data must be preprocessed to ready it for use in training the neural network. This process may involve converting the board positions into a format that the neural network can interpret, such as a tensor or a flattened vector. Additionally, one-hot encoding can be employed to represent the moves as categorical variables.

The primary objective of building a neural network is to establish multiple examples of the independent variable "x," which encodes the board-state, and the dependent outputs "y_policy" and "y_value_head." The former suggests the optimal move for the current player, while the latter indicates the present evaluation of the position (-1 for black win, +1 for white win, and any value in between for ongoing games). This evaluation can be thought of as the stockfish eval bar, which is commonly used.

This Jupyter Notebook offers different approaches for generating training data that can be used to train a neural network using several techniques.

### 1st approach: Get publicly available lichess data.

 If you're interested in downloading an official torrent that contains records of all standard games played in a month on lichess, you can join the website https://database.lichess.org/. These records typically include around 100 million games, with almost 6% (for December 2022) of them having been analyzed by Stockfish. The Stockfish evaluation for each move is also appended as a comment on the game's mainlines. In order to create our dataset, we only choose games that have been analyzed by Stockfish, meaning that they include Stockfish evaluations as well.

In [5]:
import nbimporter
from Board_to_NeuralNetwork import board_to_network_input
from Board_to_NeuralNetwork import all_possible_moves
from MCTS_and_selfplay import choose_files_with_pattern
import numpy as np
import chess


possible_moves=all_possible_moves()



This code defines a function separate_evaluated_games() that reads in a PGN (Portable Game Notation) file containing chess games and writes out only those games that contain an "eval" annotation to a new PGN file. The purpose of this function is to extract a subset of evaluated games from a larger PGN file for further analysis.

### 4 Vito


Maybe you could make it possible to look only at games which are played between players above 2k elo? 



In [2]:
import chess
import chess.pgn

def separate_evaluated_games(max_eval_games=10):
    # Path to input PGN file
    file = r'C:\\Users\\45819\\Downloads\\lichess_db_standard_rated_2022-12.pgn'
    
    # Counters for evaluated games and total games
    num_eval_games = 0
    num_total_games = 0
    
    # Open the input PGN file for reading
    with open(file, "r") as input_file:
        # Read games one by one from the input file until there are no more games or the max number of evaluated games is reached
        while True and num_eval_games < max_eval_games:
            # Read in the next game from the input file using the chess.pgn module
            pgn = chess.pgn.read_game(input_file)
            num_total_games += 1
            
            # If there are no more games to read, break out of the loop
            if pgn is None:
                break
            
            # Convert the game to a string with header, moves, and evaluations
            pgn_string = pgn.accept(chess.pgn.StringExporter(headers=True, variations=False, comments=True))
            
            # If the game contains an "eval" annotation which means that it has been analyzed by stockfish,
            #write it to the output file and increment the counter
            if 'eval' in pgn_string:
                num_eval_games += 1
                if num_eval_games % 10000 == 1:
                    print('Found %d games with eval from a total of %d games.' % (num_eval_games, num_total_games))
                
                # Open the output file for writing and write the game to it
                with open("output.pgn", "a") as output_file:
                    pgn_string = '\n' + pgn_string + '\n'
                    output_file.write(pgn_string)

    # Return 0 as a placeholder value
    return 0

# Uncomment the following line to call the function and extract evaluated games from the input PGN file
# separate_evaluated_games(max_eval_games=1000000)

This code defines a function called create_training_data_from_evaluated_games which reads and processes evaluated chess games stored in a PGN file (output.pgn) to create training data for a neural network to learn how to play chess. The function starts by importing necessary modules such as chess.pgn, re, and numpy. It then initializes containers for storing the training data including chess board positions, best moves played, probability of best move, and evaluation of board position. The function iterates through a maximum number of games specified by max_num_games and for each game, it reads game data from the PGN file and stores moves and evaluations from the game. The evaluation of each position is obtained from Stockfish and the board position, best move played, and probability vector are stored in the initialized containers. The evaluation scores are normalized to a range of (-1, 1) before being stored. If the game ended due to time-out, it is skipped. Finally, the output data is transformed to numpy arrays and saved in separate files.

In [3]:
import chess.pgn
import re
import numpy as np

# Function to create training data from evaluated chess games
def create_training_data_from_evaluated_games(max_num_games=25000):
    
    # Open the PGN file containing the evaluated games
    pgn = open("output.pgn")
    
    # Get the string of all possible moves using a separate function all_possible_moves()
    possible_moves = all_possible_moves()
    
    # Initialize containers for training data
    positions = [] # chess board positions
    best_moves = [] # best moves played
    best_moves_probs = [] # probability of best move
    outcomes = [] # evaluation of board position
    
    # Loop through a maximum number of games to create training data from each game
    for _ in range(max_num_games):
        
        # Read a game from the PGN file
        game = chess.pgn.read_game(pgn)
        
        # Print progress after every 100 games
        if _ % 100 == 0:
            print(_ / max_num_games * 100, '%%')
        
        # Access the game result to determine the winner
        result = game.headers["Result"]
        
        # Check how the game terminated
        mode = game.headers.get("Termination")
        
        # If the game ended normally (checkmate or draw and not due to time out)
        if mode == 'Normal':
            
            # Initialize the chess board
            board = chess.Board()
            
            # Store all moves and evaluations from the games
            moves = [str(move) for move in game.mainline_moves()]
            evals = [str(boards.eval()) for boards in game.mainline()]

            # Iteratively go through every move of the game and save training data from it
            for i, m in enumerate(moves):
                
                # Get the evaluation of the current position from Stockfish
                evaluation = evals[i]
                
                # If the game has already ended in a checkmate
                if board.is_checkmate():
                    print('board is checkmate')
                    break
                
                # If Stockfish predicts checkmate in a few moves, convert it to a float32 score
                elif '#+' in evaluation:
                    moves_to_checkmate = re.findall(r'\d+', evaluation)
                    temp = 10000. - float(100 * int(moves_to_checkmate[0]))
                elif '#-' in evaluation:
                    moves_to_checkmate = re.findall(r'\d+', evaluation)
                    temp = -10000. + float(100 * int(moves_to_checkmate[0]))
                
                # Otherwise, append the evaluation as is
                elif evaluation == 'None':
                    break
                else:
                    temp = float(evaluation)
                
                # Normalize the evaluation score to a range of (-1, 1)
                temp /= 10000    
                outcomes.append(temp)
                
                # Store the current chess board position as input
                positions.append(board_to_network_input(board))
                
                # Store the current move played as best move
                best_moves.append(m)
                
                # Convert the best move to a probability vector with 0 on all other elements and 1 on played move
                # The purpose of this initialization is to train the neural network to play like a human,
                idx = np.where(possible_moves == m)
                probs = np.zeros_like(possible_moves, dtype=np.float32)
                probs[idx] = 1.
                
                # Update the move probabilities
                best_moves_probs.append(probs)

                # Create a move object that python-chess recognises and update the board with that move
                move = chess.Move.from_uci(m)
                board.push(move)

        # If the game ended due to time out, skip it
        elif mode =='Time forfeit':
    #         print('noob got flagged\n ')
            continue

    # Close the file
    pgn.close()


    # Transform output data to numpy arrays

    positions=np.array(positions)
    best_moves=np.array(best_moves)
    best_moves_probs=np.array(best_moves_probs)
    outcomes=np.array(outcomes)
    np.save('SL_boards%0.e'% (max_num_games), positions)
    np.save('SL_best_moves%0.e' %max_num_games, best_moves)
    np.save('SL_best_moves_probs%0.e'% (max_num_games), best_moves_probs)
    np.save('outcomes%0.e' %max_num_games, outcomes)


### Generating a Dataset for Chess AI Training: Approach 2
In this approach, we generate a number of chess positions by randomly sequencing moves and then ask Stockfish for the best move and evaluation for each position. While this method can create a dataset, it has two important drawbacks:

1. The chess positions generated from a random sequence of moves may not be representative of positions that naturally arise on the board through regular gameplay.
2. The optimal move and evaluation calculations performed by Stockfish can be time-consuming and create a bottleneck in the dataset creation process.
Due to these reasons, this method may not be the most effective for creating a dataset for neural network training. However, it can be useful for generating end-game positions, which are typically the weakness of policy head decisions.

To facilitate various chess-related tasks using the Stockfish engine, we have provided two functions: stockfish() and stockfish_best_move().

The stockfish() function takes a chess board position represented by a chess.Board() object and analyzes it using the Stockfish chess engine to obtain an assessment of the game's score for the current board position, measured in centipawns (100 centipawns ~ 1 pawn). The evaluation score generated by the Stockfish engine ranges from -10000 to +10000 centipawns.

The function can be called with the following parameters:

board: a chess.Board() object representing the chess board position.
depth: the maximum search depth that the Stockfish engine will use to analyze the board. If None, the engine will analyze the position until a final score is obtained.
time_limit: the maximum amount of time in seconds that the Stockfish engine will use to analyze the board. If None, the engine will analyze the position until a final score is obtained. The user can choose to use either depth or time_limit, but not both.
The stockfish_best_move() function also takes a chess board position represented by a chess.Board() object and uses the Stockfish engine to find the best move for the current board position. The function can be called with the following parameters:

board: a chess.Board() object representing the chess board position.
time_limit: the maximum amount of time in seconds that the Stockfish engine will use to find the best move for the current board position. The default value is 1.0.
Note that these functions rely on a specific path to the Stockfish executable (r'C:\Users\45819\Downloads\chess\stockfish_15\stockfish_15_x64.exe'), which may not be compatible with your system. You may need to adjust the path based on the location of the Stockfish engine on your system. If you do not have Stockfish installed, you can download it from the official website at https://stockfishchess.org/download/windows/.



The stockfish_best_move() function takes a chess board position and uses the Stockfish engine to find the best move for the current board position. The function can be called with the following parameters:

board: a chess board position represented by a chess.Board() object.
time_limit: the maximum amount of time in seconds that the Stockfish engine will use to find the best move for the current board position. The default value is 1.0.
Upon execution, the stockfish_best_move() function opens the Stockfish engine, finds the best move for the current board position, prints it, and closes the engine. The function returns the best move as a chess.Move() object.

Note that these functions are dependent on a specific path to the Stockfish executable (r'C:\Users\45819\Downloads\chess\stockfish_15\stockfish_15_x64.exe') which may not be compatible with your system. As such, you may need to adjust the path accordingly based on the location of the Stockfish engine on your system. If you do not have Stockfish installed on your system, you can download it from the official website at https://stockfishchess.org/download/windows/.

In [4]:

import random
import chess
import chess.engine
def stockfish(board, depth=None,time_limit=None): 
    file_name = r'C:\Users\45819\Downloads\chess\stockfish_15\stockfish_15_x64.exe'
    with chess.engine.SimpleEngine.popen_uci(file_name) as sf: # Start the chess engine
        if time_limit==None:
            result = sf.analyse(board, chess.engine.Limit(depth=depth)) # Analyze the position
            score1 = result['score'].white().score() # Get the score for the position from white's perspective
        elif depth==None:
            result = sf.analyse(board, chess.engine.Limit(time=time_limit)) # Analyze the position
            score1 = result['score'].white().score() # Get the score for the position from white's perspective
    return (score1)
    
def stockfish_best_move(board,time_limit=1.):
    file_name = r'C:\Users\45819\Downloads\chess\stockfish_15\stockfish_15_x64.exe'
    # Create a chess board
#     board = chess.Board()

    # Start the chess engine
    engine = chess.engine.SimpleEngine.popen_uci(file_name)

    # Find the best move for the current board position
    result = engine.play(board, chess.engine.Limit(time=time_limit))

    # Print the best move
    print("Best move:", result.move)

    # Close the engine
    engine.quit()
    return result.move



The function random_board(max_depth=200) generates a random chess board position by making a random sequence of legal moves on a new chess board. The function can be called with the following parameter:

max_depth: the maximum number of moves to make. A random number of moves between 0 and max_depth is generated.
The function works as follows:

It starts by creating a new chess board object using chess.Board().
It then generates a random number of moves between 0 and max_depth using random.randrange(0, max_depth).
Next, it enters a loop that iterates depth times, where depth is the random number of moves generated in the previous step.
In each iteration of the loop, it gets a list of all legal moves using board.legal_moves, chooses a random move from the list using random.choice(all_moves), applies the chosen move to the board using board.push(random_move), and checks if the game is over using board.is_game_over().
If the game is over, the function stops making moves and returns the final board position using return board.
If the game is not over, the function continues making moves until it has made the specified number of moves or until the game is over.

In [5]:
# Define a function that creates a random chess board by making a random number of legal moves
def random_board(max_depth=200):
    board = chess.Board() # Create a new chess board
    depth = random.randrange(0, max_depth) # Generate a random number of moves to make

    # Make the specified number of moves
    for _ in range(depth):
        all_moves = list(board.legal_moves) # Get a list of all legal moves
        random_move = random.choice(all_moves) # Choose a random move from the list
        board.push(random_move) # Apply the move to the board
        if board.is_game_over(): # If the game is over, stop making moves
            break

    return board # Return the final board

In [2]:
# Create a chess board object
def set_network_probs(board,move):
    idx=np.where(possible_moves==move) #Find the indexes of legal moves
    probs=np.zeros_like(possible_moves,dtype=np.float32) # Initialize the policy head
    probs[idx]=1. # Set a probability of 1 for the move suggested by stockfish, and 0 for all other moves
    # Return the move probabilities.
    return probs


In [3]:
# A function which returns the material remaining on board for white and black:

def material(board):
    # Define a dictionary of piece values
    piece_values = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 0
    }

    # Initialize variables to store material values
    white_material = 0
    black_material = 0

    # Get a dictionary of all the pieces on the board along with their positions
    piece_map = board.piece_map()

    # Iterate through the dictionary to calculate total material for each color
    for square, piece in piece_map.items():
        if piece.color == chess.WHITE:
            white_material += piece_values.get(piece.piece_type, 0)
        else:
            black_material += piece_values.get(piece.piece_type, 0)

    return white_material,black_material


### Time to create some endgame positions 

The reasons for making this dataset is that the neural network is failing to make solid choices in the end-game. Therefore we supply it with more examples of endgame positions data and best moves to cover better the parameter-space of the input data(positions).
This function creates an endgame dataset by generating random chess board positions and collecting their corresponding stockfish evaluations and best moves. It saves the resulting dataset as numpy arrays in separate files for the inputs, evaluations, and move probabilities.

In [8]:
import numpy as np

# This function generates a dataset of chess endgame positions, their evaluations, and move probabilities
def create_endgame_dataset(N_dataset, time_limit):
    # Initialize empty lists to store inputs (x), evaluations (y1), and move probabilities (y2)
    x=[]
    y1=[]
    y2=[]
    
    # Initialize a counter variable for the number of generated positions
    i=0
    
    # Loop until the desired number of positions is generated
    while i < N_dataset:
        # Generate a random chess board with a maximum depth of 600
        board = random_board(max_depth=600)
        
        # Calculate the material score for both players
        wm, bm = material(board)
        
        # Only consider positions where the total material on the board is less than 40 (endgame)
        if wm + bm < 40:
            try:
                # Print the progress of generating the dataset every 10 saved positions
                if i % 10 == 0:
                    print(i * 100 / N_dataset, '%% progress')
                
                # Evaluate the board using Stockfish and store the score
                score = stockfish(board, time_limit=time_limit, stockfish_mode='one player')
                
                # Get the best move for the current board
                best_move = stockfish_best_move(board, time_limit=time_limit)
                
                # Normalize the score by dividing it by 10000 and append the input, evaluation, and move probabilities to their respective lists
                saved_score = float(score) / 10000
                x.append(board_to_network_input(board))
                y1.append(saved_score)
                y2.append(set_network_probs(board, str(best_move)))
                
                # Increment the position counter
                i += 1
                
                # Save the current dataset as numpy arrays in separate files for the inputs, evaluations, and move probabilities
                num_output = len(choose_files_with_pattern(max_num=9999999, pickles_path='./endgame_data/positions*')) + 1
                formatted_string = "{:06d}".format(num_output)
                string1 = r'C:\Users\45819\Downloads\chess\chess\endgame_data\positions_' + formatted_string
                string2 = r'C:\Users\45819\Downloads\chess\chess\endgame_data\evaluations_' + formatted_string
                string3 = r'C:\Users\45819\Downloads\chess\chess\endgame_data\move_probs_' + formatted_string
                np.save(string1, x)
                np.save(string2, y1)
                np.save(string3, y2)
                
                # Clear the input, evaluation, and move probability lists
                x = []
                y1 = []
                y2 = []
            except Exception as e:
                # If an error occurs, print the error message and continue generating positions
                print(f"An error occurred: {e}")
                print('whoops!')
                continue
    
    # Return the inputs, evaluations, and move probabilities for the generated positions
    return x, y1, y2

# Example usage of the function
# x, y1, y2 = create_endgame_dataset(N_dataset=1, time_limit=0.25)
# print(np.shape(x), np.shape(y1), np.shape(y2))

 Note that the user sets a maximum number of positions and a time limit for the stockfish evaluation for each of  these positions, but the code saves incrementally the positions in seperate files. This is of particular use when the time of the calculation has been underestimated, and the ending needs to stop without having saved any position. In order to pass this data to the neural netwrok it is somewhat convenient to consolidate this dataset which is split among manydata to a single data file. For this reason the next cell uses some basic np functions in order to stack the position arrays in a single array

In [ ]:
###### import time
import numpy as np

# Set the maximum number of files to process
n = 2

# Get a list of file names that match a specific pattern in the directory
position_names = choose_files_with_pattern(max_num=n, pickles_path='./endgame_data/positions*')

# Sort the file names in alphabetical order
names = sorted(position_names)

# Generate the names of the corresponding evaluation and move probability files
evaluation_names = [string.replace("positions", "evaluations") for string in names]
moves_probs_names = [string.replace("positions", "move_probs") for string in names]

# Load the position, move probability, and evaluation arrays from the files
positions_s = [np.load(position_names[i]) for i in range(len(position_names))]
move_probs = [np.load(moves_probs_names[i]) for i in range(len(moves_probs_names))]
evaluations = [np.load(evaluation_names[i]) for i in range(len(evaluation_names))]

# Stack the position arrays into a single array
positions = np.stack(positions_s)
move_probs=np.stack(move_probs)
evaluations=np.stack(evaluations)

#Squeeze the unnecessary dimension
positions=np.squeeze(positions,axis=1)
evaluations=np.squeeze(evaluations,axis=1)
move_probs=np.squeeze(move_probs,axis=1)


# Uncomment this line to save the end-game data.
# np.save('final_endgame_pos',positions)
# np.save('final_endgame_move_probs',move_probs)
# np.save('final_endgame_evaluations',evaluations)

### 3rd approach: Create a dataset through chess puzzles!

The third option is a nice alternative to the previous cumbersome computationally approach. Here lichess savees us once again , since htey host a torrent which contains  3 milion chess puzzles along with their solutions. This can be found in https://database.lichess.org/#puzzles and downloaded as a CSV file which is read in the lines below. The general idea of this approach is to build a dataset which does not contain of predicted position evaluation, but only the policy head, the optimal move for each position. 

In [9]:
#Load the data from the CSV file:

import csv

puzzle_ids=[]
fens=[]
ratings=[]
moves_solution=[]
games_url=[]
j=0
with open( r'C:\\Users\\45819\\Downloads\\lichess_db_puzzle.csv', "r") as file:
    reader = csv.reader(file)
    for row in reader:
        puzzle_id = row[0]
        puzzle_ids.append(puzzle_id)
        j+=1
        fen = row[1]
        fens.append(fen)
        
        moves = row[2]
        moves_solution.append(moves)
        
        rating = row[3]
        ratings.append(rating)
        rating_deviation = row[4]
        popularity = row[5]
        nb_plays = row[6]
        themes = row[7]
        game_url = row[8]
        games_url.append(game_url)
        opening_family = row[9]
        
print(j,' chess puzzles found')

3080529


In [14]:
# Save the data in a neural network friendly way:
import chess
import time
import numpy as np

t0=time.time()
x=[]
y_policy=[]
N_puzzles=500000
for i in range(1,N_puzzles):
    if i%100==0:
        t2=time.time()
        remaining=N_puzzles-i
        puzz_p_s=i/(t2-t0)
        t=remaining/puzz_p_s
        hours = t // 3600
        remaining_seconds = t % 3600
        minutes = remaining_seconds // 60
        seconds = remaining_seconds % 60
        print(f"{hours} hour(s), {minutes} minute(s), and {seconds} second(s).")
    fen=fens[i]
    moves=moves_solution[i]
    rating=ratings[i]
    game_url=games_url[i]
#     print(game_url)
#     print(moves)
#     print(' This puzzle has a rating of ',rating)

    board=chess.Board()
    board.set_fen(fen)
#     print('starting board:\n',board)
    for j,move in enumerate(moves.split()):
#         print(board)
#         print(move)

        if j>0:
            x.append(board_to_network_input(board))
#             print('saving board: \n',board)
#             print('saving move:',move)
            probs=set_network_probs(board,move)
            y_policy.append(probs)
            
        move_uci=chess.Move.from_uci(move)
        board.push(move_uci)

x=np.array(x)
y=np.array(y_policy)

print(np.shape(x))
# .
print(np.shape(y))
np.save('x_puzzles1m',x)
np.save('y_puzzles1m',y)

2.0 hour(s), 2.0 minute(s), and 16.0502347946167 second(s).
1.0 hour(s), 55.0 minute(s), and 17.356242656706854 second(s).
1.0 hour(s), 53.0 minute(s), and 44.80323529243378 second(s).
1.0 hour(s), 53.0 minute(s), and 25.36608648300171 second(s).
1.0 hour(s), 52.0 minute(s), and 48.163644790650324 second(s).
1.0 hour(s), 53.0 minute(s), and 9.576225360234275 second(s).
1.0 hour(s), 53.0 minute(s), and 0.9147597721648708 second(s).
1.0 hour(s), 51.0 minute(s), and 58.491485595703125 second(s).
1.0 hour(s), 52.0 minute(s), and 24.59758196936764 second(s).
1.0 hour(s), 52.0 minute(s), and 19.729464292526245 second(s).
1.0 hour(s), 52.0 minute(s), and 45.01139213822171 second(s).
1.0 hour(s), 53.0 minute(s), and 45.371468623479814 second(s).
1.0 hour(s), 53.0 minute(s), and 21.720283031463623 second(s).
1.0 hour(s), 53.0 minute(s), and 52.23837835448194 second(s).
1.0 hour(s), 53.0 minute(s), and 40.2382672627773 second(s).
1.0 hour(s), 53.0 minute(s), and 43.913697600364685 second(s).
1.0

1.0 hour(s), 52.0 minute(s), and 38.608319277153896 second(s).
1.0 hour(s), 52.0 minute(s), and 40.29797988151404 second(s).
1.0 hour(s), 52.0 minute(s), and 39.4159471988678 second(s).
1.0 hour(s), 52.0 minute(s), and 38.70537387623517 second(s).
1.0 hour(s), 52.0 minute(s), and 34.96412193514061 second(s).
1.0 hour(s), 52.0 minute(s), and 36.08937964232064 second(s).
1.0 hour(s), 52.0 minute(s), and 32.591916082574244 second(s).
1.0 hour(s), 52.0 minute(s), and 33.19077546255903 second(s).
1.0 hour(s), 52.0 minute(s), and 26.792744918918288 second(s).
1.0 hour(s), 52.0 minute(s), and 22.45367251315656 second(s).
1.0 hour(s), 52.0 minute(s), and 19.85678062572333 second(s).
1.0 hour(s), 52.0 minute(s), and 19.06620416376336 second(s).
1.0 hour(s), 52.0 minute(s), and 16.39028549194336 second(s).
1.0 hour(s), 52.0 minute(s), and 14.764654368570518 second(s).
1.0 hour(s), 52.0 minute(s), and 14.142275805376812 second(s).
1.0 hour(s), 52.0 minute(s), and 12.25346856503893 second(s).
1.0 

1.0 hour(s), 49.0 minute(s), and 59.611673638505636 second(s).
1.0 hour(s), 49.0 minute(s), and 58.014077597094 second(s).
1.0 hour(s), 49.0 minute(s), and 58.299013583401575 second(s).
1.0 hour(s), 49.0 minute(s), and 57.74004684277406 second(s).
1.0 hour(s), 49.0 minute(s), and 55.62533350504873 second(s).
1.0 hour(s), 49.0 minute(s), and 56.00633662718337 second(s).
1.0 hour(s), 49.0 minute(s), and 56.199010450461174 second(s).
1.0 hour(s), 49.0 minute(s), and 51.681244716924084 second(s).
1.0 hour(s), 49.0 minute(s), and 49.0941733964637 second(s).
1.0 hour(s), 49.0 minute(s), and 49.03710202579077 second(s).
1.0 hour(s), 49.0 minute(s), and 48.0238256237717 second(s).
1.0 hour(s), 49.0 minute(s), and 48.36099951509095 second(s).
1.0 hour(s), 49.0 minute(s), and 46.1590994144608 second(s).
1.0 hour(s), 49.0 minute(s), and 45.62954186707066 second(s).
1.0 hour(s), 49.0 minute(s), and 43.72146123882703 second(s).
1.0 hour(s), 49.0 minute(s), and 42.525992938450145 second(s).
1.0 hour

1.0 hour(s), 47.0 minute(s), and 2.6159527884319687 second(s).
1.0 hour(s), 47.0 minute(s), and 2.0657827183231348 second(s).
1.0 hour(s), 47.0 minute(s), and 0.9454934830055208 second(s).
1.0 hour(s), 46.0 minute(s), and 59.67416489124207 second(s).
1.0 hour(s), 46.0 minute(s), and 57.71327093117179 second(s).
1.0 hour(s), 46.0 minute(s), and 55.97298006038727 second(s).
1.0 hour(s), 46.0 minute(s), and 54.16614604587903 second(s).
1.0 hour(s), 46.0 minute(s), and 54.12045276283061 second(s).
1.0 hour(s), 46.0 minute(s), and 50.939076797461894 second(s).
1.0 hour(s), 46.0 minute(s), and 48.36752718657681 second(s).
1.0 hour(s), 46.0 minute(s), and 46.67204795598445 second(s).
1.0 hour(s), 46.0 minute(s), and 45.46267512265331 second(s).
1.0 hour(s), 46.0 minute(s), and 43.63427313324246 second(s).
1.0 hour(s), 46.0 minute(s), and 42.24521395055217 second(s).
1.0 hour(s), 46.0 minute(s), and 39.62673859584629 second(s).
1.0 hour(s), 46.0 minute(s), and 38.38570321184943 second(s).
1.0 

1.0 hour(s), 43.0 minute(s), and 55.137855044384196 second(s).
1.0 hour(s), 43.0 minute(s), and 53.87636735754131 second(s).
1.0 hour(s), 43.0 minute(s), and 51.88983615463985 second(s).
1.0 hour(s), 43.0 minute(s), and 50.24210461637995 second(s).
1.0 hour(s), 43.0 minute(s), and 48.981220041833694 second(s).
1.0 hour(s), 43.0 minute(s), and 47.37660431683253 second(s).
1.0 hour(s), 43.0 minute(s), and 45.77706270797171 second(s).
1.0 hour(s), 43.0 minute(s), and 44.2825685401458 second(s).
1.0 hour(s), 43.0 minute(s), and 43.80334583252352 second(s).
1.0 hour(s), 43.0 minute(s), and 42.47359723761201 second(s).
1.0 hour(s), 43.0 minute(s), and 40.92814109144001 second(s).
1.0 hour(s), 43.0 minute(s), and 39.94556983311941 second(s).
1.0 hour(s), 43.0 minute(s), and 38.614733661609534 second(s).
1.0 hour(s), 43.0 minute(s), and 36.707338436943246 second(s).
1.0 hour(s), 43.0 minute(s), and 34.96637997003745 second(s).
1.0 hour(s), 43.0 minute(s), and 33.63679161141863 second(s).
1.0 h

...

0.0 hour(s), 14.0 minute(s), and 44.63726643315988 second(s).
0.0 hour(s), 14.0 minute(s), and 43.260257393149004 second(s).
0.0 hour(s), 14.0 minute(s), and 41.86631540307201 second(s).
0.0 hour(s), 14.0 minute(s), and 40.48213017582066 second(s).
0.0 hour(s), 14.0 minute(s), and 39.09299594258073 second(s).
0.0 hour(s), 14.0 minute(s), and 37.69922803653458 second(s).
0.0 hour(s), 14.0 minute(s), and 36.31473042965479 second(s).
0.0 hour(s), 14.0 minute(s), and 34.928342311278584 second(s).
0.0 hour(s), 14.0 minute(s), and 33.54620127933197 second(s).
0.0 hour(s), 14.0 minute(s), and 32.163906019757064 second(s).
0.0 hour(s), 14.0 minute(s), and 30.76291040253261 second(s).
0.0 hour(s), 14.0 minute(s), and 29.3767547443897 second(s).
0.0 hour(s), 14.0 minute(s), and 27.9770353862217 second(s).
0.0 hour(s), 14.0 minute(s), and 26.60204795335278 second(s).
0.0 hour(s), 14.0 minute(s), and 25.208346995311672 second(s).
0.0 hour(s), 14.0 minute(s), and 23.81224653785125 second(s).
0.0 ho

0.0 hour(s), 11.0 minute(s), and 41.25848776612588 second(s).
0.0 hour(s), 11.0 minute(s), and 39.860392842852775 second(s).
0.0 hour(s), 11.0 minute(s), and 38.46883317637969 second(s).
0.0 hour(s), 11.0 minute(s), and 37.07851514428285 second(s).
0.0 hour(s), 11.0 minute(s), and 35.69154864418476 second(s).
0.0 hour(s), 11.0 minute(s), and 34.29501048723864 second(s).
0.0 hour(s), 11.0 minute(s), and 32.899284416345836 second(s).
0.0 hour(s), 11.0 minute(s), and 31.507575902659255 second(s).
0.0 hour(s), 11.0 minute(s), and 30.123813959696122 second(s).
0.0 hour(s), 11.0 minute(s), and 28.735863395101433 second(s).
0.0 hour(s), 11.0 minute(s), and 27.339551518415988 second(s).
0.0 hour(s), 11.0 minute(s), and 25.951723769035993 second(s).
0.0 hour(s), 11.0 minute(s), and 24.565097087392928 second(s).
0.0 hour(s), 11.0 minute(s), and 23.176613834756836 second(s).
0.0 hour(s), 11.0 minute(s), and 21.780382397275844 second(s).
0.0 hour(s), 11.0 minute(s), and 20.38951420572539 second(s)

0.0 hour(s), 8.0 minute(s), and 36.47861305515414 second(s).
0.0 hour(s), 8.0 minute(s), and 35.09134465078728 second(s).
0.0 hour(s), 8.0 minute(s), and 33.70326978187063 second(s).
0.0 hour(s), 8.0 minute(s), and 32.32141394796395 second(s).
0.0 hour(s), 8.0 minute(s), and 30.94349590031385 second(s).
0.0 hour(s), 8.0 minute(s), and 29.545456645762954 second(s).
0.0 hour(s), 8.0 minute(s), and 28.15298758007924 second(s).
0.0 hour(s), 8.0 minute(s), and 26.760247218956067 second(s).
0.0 hour(s), 8.0 minute(s), and 25.364515145551195 second(s).
0.0 hour(s), 8.0 minute(s), and 23.974969239694587 second(s).
0.0 hour(s), 8.0 minute(s), and 22.583477287674896 second(s).
0.0 hour(s), 8.0 minute(s), and 21.18997886160838 second(s).
0.0 hour(s), 8.0 minute(s), and 19.803578002699453 second(s).
0.0 hour(s), 8.0 minute(s), and 18.420030281477466 second(s).
0.0 hour(s), 8.0 minute(s), and 17.03128185882389 second(s).
0.0 hour(s), 8.0 minute(s), and 15.650308832279109 second(s).
0.0 hour(s), 8.0

0.0 hour(s), 5.0 minute(s), and 30.3980372486771 second(s).
0.0 hour(s), 5.0 minute(s), and 29.008368574218935 second(s).
0.0 hour(s), 5.0 minute(s), and 27.619158892947837 second(s).
0.0 hour(s), 5.0 minute(s), and 26.22556467446549 second(s).
0.0 hour(s), 5.0 minute(s), and 24.839531873686212 second(s).
0.0 hour(s), 5.0 minute(s), and 23.450117334270203 second(s).
0.0 hour(s), 5.0 minute(s), and 22.063628031903477 second(s).
0.0 hour(s), 5.0 minute(s), and 20.67315756227265 second(s).
0.0 hour(s), 5.0 minute(s), and 19.28303540327761 second(s).
0.0 hour(s), 5.0 minute(s), and 17.901338528397446 second(s).
0.0 hour(s), 5.0 minute(s), and 16.510437927222085 second(s).
0.0 hour(s), 5.0 minute(s), and 15.119510339736564 second(s).
0.0 hour(s), 5.0 minute(s), and 13.731796701332996 second(s).
0.0 hour(s), 5.0 minute(s), and 12.348181913036285 second(s).
0.0 hour(s), 5.0 minute(s), and 10.965698143145914 second(s).
0.0 hour(s), 5.0 minute(s), and 9.577179989955482 second(s).
0.0 hour(s), 5

0.0 hour(s), 2.0 minute(s), and 24.37684133512522 second(s).
0.0 hour(s), 2.0 minute(s), and 22.98765591750322 second(s).
0.0 hour(s), 2.0 minute(s), and 21.598666012360155 second(s).
0.0 hour(s), 2.0 minute(s), and 20.210548119876876 second(s).
0.0 hour(s), 2.0 minute(s), and 18.824467367055462 second(s).
0.0 hour(s), 2.0 minute(s), and 17.435515721071454 second(s).
0.0 hour(s), 2.0 minute(s), and 16.04797937158952 second(s).
0.0 hour(s), 2.0 minute(s), and 14.659795067783477 second(s).
0.0 hour(s), 2.0 minute(s), and 13.271863887593952 second(s).
0.0 hour(s), 2.0 minute(s), and 11.882188783144016 second(s).
0.0 hour(s), 2.0 minute(s), and 10.493572522026142 second(s).
0.0 hour(s), 2.0 minute(s), and 9.106785656795097 second(s).
0.0 hour(s), 2.0 minute(s), and 7.72129931834624 second(s).
0.0 hour(s), 2.0 minute(s), and 6.331558496700964 second(s).
0.0 hour(s), 2.0 minute(s), and 4.9433817028270965 second(s).
0.0 hour(s), 2.0 minute(s), and 3.55637384805776 second(s).
0.0 hour(s), 2.0 

### 4th approach: Self-play!


Ok fellas, let's consider that we have ssome sort neural network configuration which is able to receive as an input a chess position and output a recommended move for this position. It doesnt need to be good but it needs to be able to give this output. This chapter is like a short introduction where the move predictions happen probabilistically from the policy probability distribution that the neural network outputs.

In [2]:
from tensorflow.keras import models
from   svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
import glob
import os
import pickle
import tensorflow as tf
import chess
import numpy as np
import nbimporter
import chess.svg

from Board_to_NeuralNetwork import board_to_network_input
from Board_to_NeuralNetwork import all_possible_moves
model=tf.keras.models.load_model(r'C:\Users\45819\Downloads\supervised_model__.h5')




def choose_files_with_pattern(max_num=9999, pickles_path=''):
    """
    Finds files with a given file path pattern and returns them in a list.
    Stops after finding the specified maximum number of files.
    """
    files = []
    i = 1
    for file in glob.glob(pickles_path):
        files.append(file)
        if i >= max_num:
            break
        i += 1
    return files

def save_image(board, i, formatted_string):
    """
    Saves an SVG image and PNG image of the chess board at the current state.
    """
    boardsvg = chess.svg.board(board=board)
    outputfile = './supervised_game' + formatted_string + '/match%03d.SVG' % i
    outputpng = './supervised_game' + formatted_string + '/match%03d.png' % i
    with open(outputfile, "w") as f:
        f.write(boardsvg)
    drawing = svg2rlg(outputfile)
    renderPM.drawToFile(drawing, outputpng, dpi=200, fmt="PNG")
    os.remove(outputfile)



class Chess_object():
    def __init__(self):
        """
        Initializes a new chess board object using the chess library.
        Also initializes a variable n_moves to 64.
        """
        self.board = chess.Board()
        # List of all possible chess moves
        self.moves_list = all_possible_moves()
        self.n_moves = len(self.moves_list)
        self.legal_moves = None
        self.legal_moves_string = None
    def find_move_index(self, move):
        """
        Given a move, return its index in the list of all possible moves.
        """
        idx = np.where(self.moves_list == move)
        return idx

    def find_index_move(self, index):
        """
        Given an index, return the corresponding move.
        """
        return self.moves_list[index]

    def printboard(self):
        """
        Print the current state of the chess board.
        """
        print(self.board)

    def setstartingposition(self):
        """
        Set the chess board to the starting position.
        """
        self.board = chess.Board()

    def applymove(self, move_str):
        """
        Apply the move on the chess board and update the turn.
        Also reset the legal_moves variable.
        """
        move = chess.Move.from_uci(move_str)
        board = self.board.copy()
        board.push(move)
        self.legal_moves = None
        self.board = board

    def generatelegalmoves(self):
        """
        Generate a list of legal moves and set it to the legal_moves variable.
        If the legal_moves variable is already set, return it without regenerating it.
        """
        moves = list(self.board.legal_moves)
        self.legal_moves = moves
        self.legal_moves_string = [str(move) for move in self.legal_moves]
        return self.legal_moves

    def getlegalmovesidxs(self):
        """
        Return the indices of the legal moves in the list of all possible moves.
        """
        legal_moves = [str(move) for move in self.board.legal_moves]
        indices = [i for i, x in enumerate(self.moves_list) if x in legal_moves]
        return indices

    def isTerminal(self):
        """
        Determine if the current chess board state is a terminal state.
        Print the outcome of the game.
        Return a tuple (bool, winner) where bool is True if the game is over,
        and winner is the winner of the game ('White'/'Black') or None if the game is a draw.
        """
        winner = None
        self.result = self.board.result()
        print('result', self.result)
        if self.result=='*':
            return False, None 
        elif self.result=='1-0':
            return True, 1
        elif self.result=='0-1':
            return True, -1
        elif self.result=='1/2-1/2':
            return True, 0

    def toNetworkInput(self):
        """
        Converts the current chess board state to a format that can be used as input to a neural network.
        The method uses the split_dims_rami function which is not defined in the code snippet provided.
        """
        x=board_to_network_input(self.board)
        return np.expand_dims(x, axis=0)
    def getmovesprobabilities(self):
        
        x=self.toNetworkInput()
        moves_prob,evaluation=model.predict(x)

        return moves_prob,evaluation


import os
import pickle
import numpy as np


def play_n_games(games=3, save=False):
    """
    Play a number of chess games and save the training data.
    :param games: The number of games to play.
    :param save: Whether to save the training data or not.
    """
    for game in range(games):
        # Initialize board
        b = Chess_object()
        # Set starting position
        b.setstartingposition()
        i = 0
        if save:
            # Define output images folder location
            x = len(choose_files_with_pattern(max_num=9999, pickles_path='./supervised_game*')) + 2
            print('found n games:', x, '\n\n\n\n\n')
            formatted_string = "{:03d}".format(x)
            game_folder = './supervised_game' + formatted_string
            if not os.path.isdir(game_folder):
                os.mkdir(game_folder)
            
        # Initializing placeholders for the saved training data
        positions = []
        best_moves_probs = []
        outcomes = []
        evaluations = []
        
        while True:
            # Transform the board to a neural network-friendly input
            x = b.toNetworkInput()
            # Get the move probabilities and evaluation for all possible moves
            probs, evaluation = b.getmovesprobabilities()
            # Reshape probabilities to (n_moves,)
            probs = probs.reshape(b.n_moves)
            # Generate all legal moves of the given chess board
            b.generatelegalmoves()
            # Get the indexes of the legal moves
            legal_moves_idx = b.getlegalmovesidxs()
            # Set the probability of illegal moves to zero
            mask = np.zeros_like(probs, dtype=bool)
            mask[legal_moves_idx] = True
            probs[~mask] = 0
            probs /= np.sum(probs)

            # Choose the next move
            # Probabilistic move based on the probabilities
            idx = np.random.choice(len(probs), p=probs)
            # Find the string representation of the best move
            bm = b.find_index_move(idx)
        
            print('%2.2f'%(probs[idx]/np.sum(probs)*100),'%% move probability')
            
            positions.append(x)
            best_moves_probs.append(probs)
            evaluations.append(evaluation)
            print(bm)
            # Make the move!
            b.applymove(bm)
            if save:
                save_image(b.board, i, formatted_string)
            i += 1
            condition, score = b.isTerminal()
            if condition:
                print('game is over', score)
                break
        print('game lasted ', i, ' moves \n\n\n')
        
        # Save the training data
        x = len(choose_files_with_pattern(max_num=9999999, pickles_path='./selfplaydata/positions*')) + 1
        formatted_string = "{:05d}".format(x)
        
        with open("./selfplaydata/positions"+formatted_string, "ab") as f:
            pickle.dump(positions, f)
        with open("./selfplaydata/best_moves_probs"+formatted_string, "ab") as f:
            pickle.dump(best_moves_probs, f)
        with open("./selfplaydata/evaluations"+formatted_string, "ab") as f:
            pickle.dump(evaluations, f)
        print('score:',score)
        for k in range(i):
            outcomes.append(score)
        with open("./selfplaydata/outcomes"+formatted_string,"ab") as f:
            pickle.dump(outcomes,f)
        
        

play_n_games(1,save=False)


    

1/1 [==============================] - 0s 455ms/step
3.38 %% move probability
g1f3
result *
1/1 [==============================] - 0s 19ms/step
16.80 %% move probability
b8c6
result *
1/1 [==============================] - 0s 20ms/step
43.36 %% move probability
g2g3
result *
1/1 [==============================] - 0s 22ms/step
76.04 %% move probability
e7e5
result *
1/1 [==============================] - 0s 19ms/step
40.78 %% move probability
f1g2
result *
1/1 [==============================] - 0s 18ms/step
70.52 %% move probability
d7d5
result *
1/1 [==============================] - 0s 19ms/step
39.61 %% move probability
d2d3
result *
1/1 [==============================] - 0s 22ms/step
97.73 %% move probability
h7h6
result *
1/1 [==============================] - 0s 22ms/step
100.00 %% move probability
e1g1
result *
1/1 [==============================] - 0s 19ms/step
99.99 %% move probability
g7g5
result *
1/1 [==============================] - 0s 19ms/step
99.84 %% move probability
c

1/1 [==============================] - 0s 18ms/step
95.47 %% move probability
e4e3
result *
1/1 [==============================] - 0s 19ms/step
99.79 %% move probability
c6d6
result *
1/1 [==============================] - 0s 23ms/step
99.46 %% move probability
f7e8
result *
1/1 [==============================] - 0s 23ms/step
55.51 %% move probability
d6f6
result *
1/1 [==============================] - 0s 19ms/step
99.67 %% move probability
e8e7
result *
1/1 [==============================] - 0s 19ms/step
18.51 %% move probability
f6e6
result *
1/1 [==============================] - 0s 19ms/step
99.99 %% move probability
e7f7
result *
1/1 [==============================] - 0s 22ms/step
90.78 %% move probability
e6c6
result *
1/1 [==============================] - 0s 20ms/step
78.65 %% move probability
f7e8
result *
1/1 [==============================] - 0s 19ms/step
87.11 %% move probability
c6d6
result *
1/1 [==============================] - 0s 20ms/step
72.97 %% move probability
e8

### 5th method. Self-play through MCTS




In [3]:
possible_moves=all_possible_moves()
model=tf.keras.models.load_model(r'C:\Users\45819\Downloads\supervised_model__.h5')

class TreeNode():
    def __init__(self,board):
        # average winrate of node
        self.M=0
        
        # amounts of wins
        self.W=0
        
        #amount of times this node has been visited
        self.V=0
        
        # Depth away from root
        self.depth=0
        
        # Variation name:
        self.variation=[]
        
        ### Each node should also have probabilities of being visited from previous position to be used for montecarlo
        self.Prior_Probability=0
        
        # Visited moves (at least one simulation has been done for that move)
        self.visitedMovesAndNodes=[]
        
        # Possible moves that have not been expanded
        self.nonVisitedLegalMoves=[]
        
        # the board
        self.board=board
        
        # The root node does not have any parents, for the creation of the children (moves), 
        # For the children upon creation we will modify the self.parent attribute
        self.parent=None
        
        for m in self.board.legal_moves:
            self.nonVisitedLegalMoves.append(m)
    #Is this a leaf node 
    def isMCTSLeafNode(self):
        return len(self.nonVisitedLegalMoves)!=0
    def isTerminalNode(self):
        return len(self.nonVisitedLegalMoves)==0 and len(self.visitedMovesAndNodes)==0

    
def get_network_probabilities(board):
    x=board_to_network_input(board)
    x=np.expand_dims(x, axis=0)
    # Predict move probabilities of the network
    probs,evaluation=model.predict(x)
    probs=probs.reshape(1968)
    legal_moves=[str(move) for move in list(board.legal_moves)]
    legal_moves_idxs = [i for i, x in enumerate(possible_moves) if x in legal_moves]
    mask=np.zeros_like(probs,dtype=bool)
    mask[legal_moves_idxs]=True
    probs[~mask]=0
    probs/=np.sum(probs)
    return probs
    
def Q_PLUS_U(node,parent):

    c_pukt=1
    P=node.Prior_Probability
    U=c_pukt*P*np.sqrt(parent.V)/(1+node.V)
    Q=node.M
    return Q+U



def select(node):
    #If the node is a leaf or a terminal node return the node itself.
    if node.isMCTSLeafNode() or node.isTerminalNode():
        print('selection has reached a leaf node')
        return node
    else:
        maxCUChild= None
        maxCUValue=-1000000

        for move, child in node.visitedMovesAndNodes:
            CUValChild=Q_PLUS_U(child,node)
            if (CUValChild>maxCUValue):
                maxCUChild=child
                maxCUValue=CUValChild
        if (maxCUChild==None):
            raise ValueError("could not identify child with best uct value")
        else:
            return (select(maxCUChild))

def expand(node):
    moveToExpand=node.nonVisitedLegalMoves.pop()
#     print('expanding move: ',moveToExpand)
    board= node.board.copy()
    probs=get_network_probabilities(board)
    
    idx=np.where(str(moveToExpand)==possible_moves)
    print('expanding move: ',str(moveToExpand))
    print('Normalized probability:',probs[idx][0])
    #Push the move on a new copied version of the board
    board.push(moveToExpand)
    childNode=TreeNode(board)
    childNode.parent=node
    childNode.Prior_Probability=probs[idx][0]
    childNode.depth=node.depth+1
    childNode.variation.append(str(moveToExpand))
    node.visitedMovesAndNodes.append((moveToExpand, childNode))
    return childNode
    
def simulate(node):
    board=node.board.copy()
    turn=board.turn
    moves=[]
    i=0
    ## Get the board of the node
    while((board.result()!= "1-0") and (board.result()!= "0-1") and (board.result()!= "1/2-1/2") ):
        
        probs=get_network_probabilities(board)
        idx=np.random.choice(len(probs),p=probs)
        print('MCTS simulation: move chosen:',possible_moves[idx])
        node.variation.append(possible_moves[idx])
        move=chess.Move.from_uci(possible_moves[idx])
        moves.append(move)
        board.push(move)
#         if i<10:
#             print(i+1,' move made:')
#             i+=1
#             print(board)
    payout=0.5
    o=board.result()
    print('moves played:',[str(move) for move in moves])
    if o=="1-0":
        payout=1
        print('white win')
    if o=="0-1":
        payout=0
        print('black win')
    if o=="1/2-1/2":
        payout=0.5
        print('its a draw')
    return payout



def backpropagate(node,payout):
    print('backpropagating a payout of ',payout,'for node with variation:',node.variation,'at depth', node.depth)
    node.M=((node.M*node.V)+payout)/(node.V+1)
    node.V=node.V+1
    node.W=node.W+payout
    print('updated node.M and node.V values:',node.M,node.V)
    if node.parent!=None:
        return backpropagate(node.parent,payout)
    

    
fen=   "r1bqkbnr/pppp1ppp/n7/4p3/2B1P3/5Q2/PPPP1PPP/RNB1K1NR w KQkq - 0 1"



def run_MCTS_step(fen,N_simulations):
    
    board=chess.Board(fen)
    root=TreeNode(board)

    for i in range(N_simulations):
        print('selection ',i,'is starting:')
        node=select(root)
        #if the selected node is a terminal we cannot expand
        # any child node. in this case, count htis as a win/draw/loss
        if (not node.isTerminalNode()):
    #         print('expansion ',i,'is starting:')
            node=expand(node)
            payout=simulate(node)
            backpropagate(node,payout)
        elif node.isTerminalNode():
            print('damn not terminal')

    #     print('simulation ',i,'is starting:')


    #     print('simulation result: ',payout)
    #     print('backpropagation ',i,'is starting:')


    root.visitedMovesAndNodes.sort(key=lambda x:x[1].M, reverse=True)
    return root
    
# root=run_MCTS_step(fen='8/8/2P5/1K6/8/8/8/k7 w - - 0 1',N_simulations=3)

In [4]:

def softmax(x):
    """Compute the softmax function for an array x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


def set_network_probs(board,move):
    idx=np.where(possible_moves==moves)
    probs=np.zeros_like(possible_moves,dtype=np.float32)
    probs[idx]=1.
    # Update the move probabilities.
    return probs        
            
def print_results(root):
    i_w=0
    i_d=0
    i_l=0
    root.visitedMovesAndNodes.sort(key=lambda x:x[1].M, reverse=True)
    for m,child in root.visitedMovesAndNodes[0:200]:
        print(('move:',m.uci(),'mean win rate %f %%:'%(100*child.M),2*child.M-1,' amount of visits',child.V,'depth:',child.depth,'moveprobability:%2.2f%%'%(100*child.Prior_Probability),'variation:',child.variation[0]) )
        
#                 print('len=',child.visitedMovesAndNodes[0:40])
        if len(child.visitedMovesAndNodes)!=0:
            child.visitedMovesAndNodes.sort(key=lambda x:x[1].M, reverse=True)
            print_results(child)
        
        
import numpy as np

def mcts_results_to_nn_input(root):
    i_w = 0
    w_moves = []
    w_rates = []
    i_d = 0
    d_moves = []
    d_rates = []
    i_l = 0
    l_moves = []
    l_rates = []
    
    root.visitedMovesAndNodes.sort(key=lambda x:x[1].M, reverse=True)
    
    for m, child in root.visitedMovesAndNodes[:200]:
        win_rate = 2 * child.M - 1
        print(f'move: {m.uci()}, mean win rate {100*child.M:.2f}%, amount of visits: {child.V}, depth: {child.depth}')
        if child.M > 0.5:
            i_w += 1
            w_moves.append(m.uci())
            w_rates.append(child.M)
        elif child.M == 0.5:
            i_d += 1
            d_moves.append(m.uci())
            d_rates.append(child.M)
        elif child.M < 0.5:
            i_l += 1
            l_moves.append(m.uci())
            l_rates.append(max(child.M,1e-4))
    
    print(f'We have {i_w} winning, {i_d} drawing, and {i_l} losing moves.')
    print(f'Winning moves: {w_moves}\nDrawing moves: {d_moves}\nLosing moves: {l_moves}')
    
    if len(w_moves) != 0:
        moves, rates = w_moves, w_rates
    elif len(d_moves) != 0:
        moves, rates = d_moves, d_rates
    elif len(l_moves) != 0:
        moves, rates = l_moves, l_rates
        
    indices = [np.where(possible_moves == elem)[0][0] for elem in moves if elem in possible_moves]
    probs = np.zeros_like(possible_moves, dtype=np.float32)
    evaluations=np.zeros_like(probs)
    for j, index in enumerate(indices):
        print(rates[j])
        probs[index] = rates[j]
        evaluations[index]=2*rates[j]-1
        
    print(np.sum(probs))
    probs/=np.sum(probs)
#     probs=softmax(probs)
    
    
    
    
    
    return probs,evaluations
    
    
# mcts_results_to_nn_input(root)

In [5]:

def play_n_MCTS_games(games=3, save=False, N_simulations=3):
    """
    Play a given number of games of chess and save the positions, best move probabilities,
    outcomes, and evaluations of each move in separate files if save is True.
    
    Parameters:
        games (int): The number of games to play.
        save (bool): If True, save the data generated from each game in separate files.
        N_simulations (int): The number of simulations to run in MCTS.
    """
    
    # Loop over each game
    for game in range(games):
        
        # Initialize the board object and set the starting position
        b = Chess_object()
        b.setstartingposition()
        
        i = 0
        
        # If saving, set the output images folder location
        if save:
            x = len(choose_files_with_pattern(max_num=9999, pickles_path='./supervised_game*')) + 2
            formatted_string = "{:03d}".format(x)
            game_folder = './supervised_game' + formatted_string
            if not os.path.isdir(game_folder):
                os.mkdir(game_folder)
            
        # Initialize the placeholders for the saved training data    
        positions = []
        best_moves_probs = []
        outcomes = []
        evaluations = []
        
        # Loop until the game is over
        while True:
            
            # Run Monte Carlo Simulations
            fen = b.board.fen()
            mcts_results = run_MCTS_step(fen=fen, N_simulations=N_simulations)
            probs, mean_outcomes = mcts_results_to_nn_input(mcts_results)
            x = b.toNetworkInput()
            
          
            # Get the move to play
            idx = np.random.choice(len(probs), p=probs)
            evaluation = mean_outcomes[idx]
            print('according to MCTS we have an evaluation of:', evaluation, 'for the randomly chosen move:', possible_moves[idx])
            
            # Find the string representation of the best move (for example 'e2e4')
            bm = b.find_index_move(idx)
            
            # Print the move probability and the best move
            print('%2.2f'%(probs[idx]/np.sum(probs)*100),'%% move probability')
            print(bm)
            
            # Save the data for the current move
            if i==0:
                positions=np.copy(x)
                best_moves_probs.append(probs)
                evaluations.append(0)
            else:    
                positions=np.concatenate((positions,x),axis=0)
                evaluations.append(evaluation_last)
                best_moves_probs.append(probs)
            
            evaluation_last=evaluation
            # Make the move!
            b.applymove(bm)
            if save:
                save_image(b.board,i,formatted_string)
            i+=1
            
            condition,score= b.isTerminal()
            if condition:
                print('game is over', score)
                break
        print('game lasted ',i,' moves \n\n\n')
        
        x=len(choose_files_with_pattern(max_num=9999, pickles_path='./selfplaydata/positions*'))+1
        formatted_string = "{:05d}".format(x)
        
        with open("./selfplaydata/positions"+formatted_string, "ab") as f:
            # Dump the new data to the file using pickle
            pickle.dump(positions, f)
        with open("./selfplaydata/best_moves_probs"+formatted_string, "ab") as f:
            # Dump the new data to the file using pickle
            pickle.dump(best_moves_probs, f)
        with open("./selfplaydata/evaluations"+formatted_string, "ab") as f:
            # Dump the new data to the file using pickle
            pickle.dump(evaluations, f)
        print('score:',score)
        for k in range(i):
            outcomes.append(score)
        with open("./selfplaydata/outcomes"+formatted_string,"ab") as f:
            pickle.dump(outcomes,f)
play_n_MCTS_games(games=1,save=True,N_simulations=50)

selection  0 is starting:
selection has reached a leaf node
1/1 [==============================] - 0s 261ms/step
expanding move:  a2a4
Normalized probability: 0.00056511274
1/1 [==============================] - 0s 21ms/step
MCTS simulation: move chosen: d7d5
1/1 [==============================] - 0s 20ms/step
MCTS simulation: move chosen: d2d4
1/1 [==============================] - 0s 19ms/step
MCTS simulation: move chosen: c8f5
1/1 [==============================] - 0s 21ms/step
MCTS simulation: move chosen: c1f4
1/1 [==============================] - 0s 20ms/step
MCTS simulation: move chosen: e7e6
1/1 [==============================] - 0s 20ms/step
MCTS simulation: move chosen: e2e3
1/1 [==============================] - 0s 23ms/step
MCTS simulation: move chosen: f8b4
1/1 [==============================] - 0s 21ms/step
MCTS simulation: move chosen: c2c3
1/1 [==============================] - 0s 19ms/step
MCTS simulation: move chosen: b4a5
1/1 [==============================] - 0s 21

1/1 [==============================] - 0s 19ms/step
MCTS simulation: move chosen: d3c3
1/1 [==============================] - 0s 18ms/step
MCTS simulation: move chosen: d5c5
1/1 [==============================] - 0s 18ms/step
MCTS simulation: move chosen: c3b3
1/1 [==============================] - 0s 18ms/step
MCTS simulation: move chosen: c5b6
1/1 [==============================] - 0s 19ms/step
MCTS simulation: move chosen: b3c4
1/1 [==============================] - 0s 19ms/step
MCTS simulation: move chosen: b6c6
1/1 [==============================] - 0s 19ms/step
MCTS simulation: move chosen: c4d3
1/1 [==============================] - 0s 18ms/step
MCTS simulation: move chosen: c6b6
1/1 [==============================] - 0s 18ms/step
MCTS simulation: move chosen: d3e2
1/1 [==============================] - 0s 19ms/step
MCTS simulation: move chosen: b6c5
1/1 [==============================] - 0s 19ms/step
MCTS simulation: move chosen: e2d3
1/1 [==============================] - 0s 1

KeyboardInterrupt: 